In [2]:
# packages
import numpy as np
import pandas as pd
import os

# database
import yfinance as yf
from sqlalchemy import create_engine, inspect

# visualisation
import seaborn as sns
import plotly.express as px
import matplotlib
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
plt.style.use('fivethirtyeight')
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm

from tabulate import tabulate
import quadprog
from scipy.optimize import minimize

/Users/jackstevenson/CQF/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# manually acquire factor data from f&f site
def generate_fama_factor_df():
    factor_df = pd.read_csv('/Users/jackstevenson/CQF/Project/F-F_Research_Data_5_Factors_2x3_daily.csv')

    factor_df['Date'] = pd.to_datetime(factor_df['Date'], format='%Y%m%d')

    factor_df = factor_df.set_index('Date')

    # align our data with start date 
    factor_df = factor_df[factor_df.index >= '2010-09-27']

    factor_df.rename(columns={
        'SMB': 'Factor SMB', # small business
        'HML': 'Factor HML', # value 
        'RMW': 'Factor RMW', # robust vs weak
        'CMA': 'Factor CMA' # conversative minus aggressive
        }, inplace=True)

    factor_df.drop(columns=['Mkt-RF', 'RF'], axis=1,  inplace=True)

    factor_df = factor_df / 100

    return factor_df

In [4]:
# create the database

# # download (only need close)
df = yf.download('IYE IYM IYJ IYC IYK IYH IYF IYW IYZ IDU IYR ^IRX ^VIX ^BCOM AGG', start='2010-09-24', end='2024-06-30')['Close']

# What are the 11 GICS sectors
# Energy - iShares U.S. Energy ETF (IYE)
# Materials - iShares U.S. Materials ETF (IYM)
# Industrials - iShares U.S. Industrials ETF (IYJ)
# Consumer Discretionary - iShares U.S. Consumer Disc ETF (IYC)
# Consumer Staples - iShares U.S. Consumer Staples ETF (IYK)
# Health Care - iShares U.S. Healthcare ETF (IYH)
# Financials - iShares U.S. Financials ETF (IYF)
# Information Technology - iShares U.S. Technology ETF (IYW)
# Communication Services - iShares U.S. Telecom ETF (IYZ)
# Utilities - iShares U.S. Utilities ETF (IDU)
# Real Estate - iShares US Real Estate ETF (IYR)

# 3M Treasury
# 13 Week Treasury Bill - ^IRX 

# Exogenous
# Volatility - CBOE Volatility Index - (^VIX)
# Commodity - Bloomberg Commodity Index Total Return (^BCOM)
# Bonds - iShares Core US Aggregate Bond ETF (AGG)

# add the factor data and concatenate
factor_df = generate_fama_factor_df()

df = pd.concat([df, factor_df], axis=1)

df.rename(columns={
    'IYE': 'Energy',
    'IYM': 'Materials',
    'IYJ': 'Industrials',
    'IYC': 'Consumer Discretionary',
    'IYK': 'Consumer Staples',
    'IYH': 'Health Care',
    'IYF': 'Financials',
    'IYW': 'Information Technology',
    'IYZ': 'Communication Services',
    'IDU': 'Utilities',
    'IYR': 'Real Estate',
    '^IRX':'3M TB',
    '^VIX': 'Volatility (Exo)',
    '^BCOM': 'Commodities (Exo)',
    'AGG': 'Bonds (Exo)'
    }, inplace=True)

# # reset index
df.reset_index(inplace=True)

# # create our engine
engine = create_engine("sqlite:///project_portfolio.db")

# # if_exits will overwrite the pre-existing table
df.to_sql('portfolio_data', engine, if_exists='replace', index=False)

[*********************100%%**********************]  15 of 15 completed


3463